<a href="https://colab.research.google.com/github/jonash-chataut/PyTorch-learnings-/blob/main/lstm_next_word_predictor_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [4]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# tokenize
tokens = word_tokenize(document.lower())

In [6]:
# build vocab
vocab={'<unk>': 0}
for token in Counter(tokens):
  if token not in vocab:
    vocab[token]=len(vocab)

In [7]:
len(vocab)

289

In [8]:
# extract sentences from data
input_sentence=document.split('\n')

In [9]:
def text_to_indices(sentence,vocab):
  numerical_sentence=[]
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [10]:
input_numerical_sentence=[]

for sentence in input_sentence:
  input_numerical_sentence.append(text_to_indices(word_tokenize(sentence.lower()),vocab))



In [11]:
len(input_numerical_sentence)

78

In [12]:
training_squence=[]

for sentence in input_numerical_sentence:
  for i in range(1,len(sentence)):
    training_squence.append(sentence[:i+1])


In [13]:
len(training_squence)

942

In [14]:
len_list=[]

for sequence in training_squence:
  len_list.append(len(sequence))
max(len_list)

62

In [15]:
padding_training_squence=[]

for sequence in training_squence:
  padding_training_squence.append([0]*(max(len_list)-len(sequence))+sequence)


In [16]:
padding_training_squence=torch.tensor(padding_training_squence,dtype=torch.long)

In [17]:
X=padding_training_squence[:,:-1]
Y=padding_training_squence[:,-1]

In [18]:
class CustomDataset(Dataset):
  def __init__(self,x,y):
    self.X=x;
    self.Y=y;

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index],self.Y[index]

In [19]:
dataset=CustomDataset(X,Y)

In [20]:
dataloader=DataLoader(dataset,batch_size=32,shuffle=True)

In [24]:
class LSTMModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,100)
    self.lstm=nn.LSTM(100,150,batch_first=True)
    self.fc=nn.Linear(150,vocab_size)

  def forward(self,x):
    embedding=self.embedding(x)
    intermediate_hidden_states,(final_hidden_state,final_cell_state)=self.lstm(embedding)
    output=self.fc(final_hidden_state.squeeze(0))
    return output



In [25]:
model=LSTMModel(len(vocab))


In [27]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [29]:
epochs=50
learning_rate=0.001
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


In [30]:
# training loop

for epoch in range(epochs):
  total_loss=0
  for batch_x,batch_y in dataloader:
    batch_x,batch_y=batch_x.to(device),batch_y.to(device)
    optimizer.zero_grad()
    output=model(batch_x)
    loss=criterion(output,batch_y)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()

  print(f"Epoch: {epoch+1}/{epochs}, Loss: {total_loss:4f}")

Epoch: 1/50, Loss: 166.141036
Epoch: 2/50, Loss: 145.859889
Epoch: 3/50, Loss: 134.067888
Epoch: 4/50, Loss: 121.461506
Epoch: 5/50, Loss: 109.174478
Epoch: 6/50, Loss: 98.006422
Epoch: 7/50, Loss: 87.548176
Epoch: 8/50, Loss: 77.887323
Epoch: 9/50, Loss: 68.575032
Epoch: 10/50, Loss: 60.373690
Epoch: 11/50, Loss: 53.059144
Epoch: 12/50, Loss: 46.499661
Epoch: 13/50, Loss: 40.991497
Epoch: 14/50, Loss: 35.756591
Epoch: 15/50, Loss: 31.327759
Epoch: 16/50, Loss: 27.651336
Epoch: 17/50, Loss: 24.089828
Epoch: 18/50, Loss: 21.421497
Epoch: 19/50, Loss: 18.966231
Epoch: 20/50, Loss: 16.877866
Epoch: 21/50, Loss: 15.149875
Epoch: 22/50, Loss: 13.749181
Epoch: 23/50, Loss: 12.453675
Epoch: 24/50, Loss: 11.491184
Epoch: 25/50, Loss: 10.557653
Epoch: 26/50, Loss: 9.861411
Epoch: 27/50, Loss: 9.214848
Epoch: 28/50, Loss: 8.645353
Epoch: 29/50, Loss: 8.077041
Epoch: 30/50, Loss: 7.740278
Epoch: 31/50, Loss: 7.464870
Epoch: 32/50, Loss: 6.877415
Epoch: 33/50, Loss: 6.739286
Epoch: 34/50, Loss: 6.

In [38]:
from IPython.core.displayhook import tokenize
# prediction

def prediction(model,vocab,text):

  # text->numerical indices
  tokenized_text=word_tokenize(text.lower())
  numerical_text=text_to_indices(tokenized_text,vocab)

  # padding
  padding_text=torch.tensor([0]*(max(len_list)-len(numerical_text))+numerical_text,dtype=torch.long).unsqueeze(0)

  # send to model
  output=model(padding_text)

  # predicted index
  value,predicted_index=torch.max(output,dim=1)
  # merge with text
  return text +" "+ list(vocab.keys())[predicted_index]


In [39]:
import time

num_token=10;
input_text="The course follows a monthly"
for i in range(num_token):
  output_text=prediction(model,vocab,input_text)
  print(output_text)
  input_text=output_text
  time.sleep(0.5)

The course follows a monthly subscription
The course follows a monthly subscription model
The course follows a monthly subscription model where
The course follows a monthly subscription model where you
The course follows a monthly subscription model where you have
The course follows a monthly subscription model where you have to
The course follows a monthly subscription model where you have to make
The course follows a monthly subscription model where you have to make monthly
The course follows a monthly subscription model where you have to make monthly payments
The course follows a monthly subscription model where you have to make monthly payments of


In [41]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 95.65%
